In [21]:
import pandas as pd
import numpy as np
import requests
import os

pd.options.mode.chained_assignment = None  # default='warn', to bypass warning

In [22]:
#File location:
#nfl_json = "espnscores.json"
nfl_json = "c:/users/itrem/OneDrive/Dev/jedha/nfl/espn_scores.json"

#Import and convert json to pd.df
df = pd.read_json(nfl_json)

#Print basic information
display(df.describe(include="all"))
display(df.head())

season         week awayteam hometeam    awayscore    homescore  \
count   1294.000000  1294.000000     1294     1294  1294.000000  1294.000000   
unique          NaN          NaN       33       33          NaN          NaN   
top             NaN          NaN  Falcons  Bengals          NaN          NaN   
freq            NaN          NaN       41       41          NaN          NaN   
mean    2019.027821     9.153014      NaN      NaN    22.496909    23.757342   
std        1.422121     5.069334      NaN      NaN    10.124419    10.312136   
min     2017.000000     1.000000      NaN      NaN     0.000000     0.000000   
25%     2018.000000     5.000000      NaN      NaN    16.000000    17.000000   
50%     2019.000000     9.000000      NaN      NaN    23.000000    24.000000   
75%     2020.000000    14.000000      NaN      NaN    30.000000    31.000000   
max     2021.000000    18.000000      NaN      NaN    59.000000    57.000000   

                                            gamecast  \
count                                           1294   
unique                                          1294   
top     https://espn.com/nfl/game/_/gameId/401030690   
freq                                               1   
mean                                             NaN   
std                                              NaN   
min                                              NaN   
25%                                              NaN   
50%                                              NaN   
75%                                              NaN   
max                                              NaN   

                                                boxscore        idgame  \
count                                               1294  1.294000e+03   
unique                                              1294           NaN   
top     https://espn.com/nfl/boxscore/_/gameId/401030690           NaN   
freq                                                   1           NaN   
mean                                                 NaN  4.011342e+08   
std                                                  NaN  1.338145e+05   
min                                                  NaN  4.009502e+08   
25%                                                  NaN  4.010308e+08   
50%                                                  NaN  4.011280e+08   
75%                                                  NaN  4.012203e+08   
max                                                  NaN  4.013266e+08   

       awayteam global record awayteam away record hometeam global record  \
count                    1294                 1294                   1294   
unique                    198                   80                    199   
top                       1-1                  0-1                    0-1   
freq                       39                   88                     38   
mean                      NaN                  NaN                    NaN   
std                       NaN                  NaN                    NaN   
min                       NaN                  NaN                    NaN   
25%                       NaN                  NaN                    NaN   
50%                       NaN                  NaN                    NaN   
75%                       NaN                  NaN                    NaN   
max                       NaN                  NaN                    NaN   

       hometeam home record  
count                  1279  
unique                   73  
top                     1-0  
freq                     87  
mean                    NaN  
std                     NaN  
min                     NaN  
25%                     NaN  
50%                     NaN  
75%                     NaN  
max                     NaN

season  week  awayteam hometeam  awayscore  homescore  \
0    2018     2    Ravens  Bengals         23         34   
1    2018     2  Panthers  Falcons         24         31   
2    2018     2  Chargers    Bills         31         20   
3    2018     2   Vikings  Packers         29         29   
4    2018     2    Texans   Titans         17         20   

                                       gamecast  \
0  https://espn.com/nfl/game/_/gameId/401030690   
1  https://espn.com/nfl/game/_/gameId/401030691   
2  https://espn.com/nfl/game/_/gameId/401030704   
3  https://espn.com/nfl/game/_/gameId/401030705   
4  https://espn.com/nfl/game/_/gameId/401030694   

                                           boxscore     idgame  \
0  https://espn.com/nfl/boxscore/_/gameId/401030690  401030690   
1  https://espn.com/nfl/boxscore/_/gameId/401030691  401030691   
2  https://espn.com/nfl/boxscore/_/gameId/401030704  401030704   
3  https://espn.com/nfl/boxscore/_/gameId/401030705  401030705   
4  https://espn.com/nfl/boxscore/_/gameId/401030694  401030694   

  awayteam global record awayteam away record hometeam global record  \
0                    1-1                  0-1                    2-0   
1                    1-1                  0-1                    1-1   
2                    1-1                  1-0                    0-2   
3                  1-0-1                0-0-1                  1-0-1   
4                    0-2                  0-2                    1-1   

  hometeam home record  
0                  1-0  
1                  1-0  
2                  0-1  
3                1-0-1  
4                  1-0

In [23]:
# Pre-processing of the NFL Json data 

# Actions:

#   - Removal boxscore column, no practical information:

df.drop(['boxscore', 'idgame', 'gamecast', "awayteam global record", "awayteam away record", "hometeam global record", "hometeam home record"], axis=1, inplace=True)

#   - Addition of analysis features:

df.insert(df.shape[1],"score_abs",df["homescore"] - df["awayscore"]) #Absolute score of the game

df.insert(df.shape[1],"winner_home", 0) #Flagging whether the home team won (1) or lose (0)

for i in range(len(df["winner_home"])):
    if df["score_abs"][i] > 0 :
        df["winner_home"][i] = 1
    elif df["score_abs"][i] < 0 :
        df["winner_home"][i] = 0
    else :
        df["winner_home"][i] = 999999999 #Value to filter and remove later on as values are not relevant vs amount of data


df.insert(df.shape[1],"winner_away", 0) #Flagging whether the away team won (1) or lose (0)

for i in range(len(df["winner_away"])):
    if df["score_abs"][i] < 0 :
        df["winner_away"][i] = 1
    elif df["score_abs"][i] > 0 :
        df["winner_away"][i] = 0
    else :
        df["winner_away"][i] = 999999999 #Value to filter and remove later on as values are not relevant vs amount of data



#   - Removal of draw result lines as they have no real impact

print("Values before draw games (value 999999999) clean up: \n")
print(df.value_counts("winner_home"))
print(df.value_counts("winner_away"))

drop_lines = df[df["score_abs"] == 0].index
df = df.drop(drop_lines, axis=0)

print("\n Values before draw games (value 999999999) clean up: \n")
print(df.value_counts("winner_home"))
print(df.value_counts("winner_away"))

#   - Removal of brackets from idgame, turn field into integer:

#df["idgame"] = df["idgame"].apply(lambda x : str(x).strip("[").strip("]").strip("'")).astype(int)


#   - Redskins changed their name to Washington. Replacing the former with the latter

df["awayteam"].replace("Redskins", "Washington", inplace=True)
df["hometeam"].replace("Redskins", "Washington", inplace=True)

#   - Reseting the index
df = df.reset_index(drop=True)


#   - Deletion of useless variables
del drop_lines

#   - Clean file creation for analysis

os.makedirs('working_files', exist_ok=True)  
df.to_csv('working_files/scores_prep.csv')  

df

Values before draw games (value 999999999) clean up: 

winner_home
1            695
0            594
999999999      5
dtype: int64
winner_away
0            695
1            594
999999999      5
dtype: int64

 Values before draw games (value 999999999) clean up: 

winner_home
1    695
0    594
dtype: int64
winner_away
0    695
1    594
dtype: int64


season  week  awayteam    hometeam  awayscore  homescore  score_abs  \
0       2018     2    Ravens     Bengals         23         34         11   
1       2018     2  Panthers     Falcons         24         31          7   
2       2018     2  Chargers       Bills         31         20        -11   
3       2018     2    Texans      Titans         17         20          3   
4       2017     2    Texans     Bengals         13          9         -4   
...      ...   ...       ...         ...        ...        ...        ...   
1284    2021    18     49ers        Rams         27         24         -3   
1285    2021    18  Patriots    Dolphins         24         33          9   
1286    2021    18  Seahawks   Cardinals         38         30         -8   
1287    2021    18  Panthers  Buccaneers         17         41         24   
1288    2021    18  Chargers     Raiders         32         35          3   

      winner_home  winner_away  
0               1            0  
1               1            0  
2               0            1  
3               1            0  
4               0            1  
...           ...          ...  
1284            0            1  
1285            1            0  
1286            0            1  
1287            1            0  
1288            1            0  

[1289 rows x 9 columns]

In [20]:
#CREATING TEAMS LIST

#Based on the games played, append both lists "away" and "home" teams even though they both should be the same

df_away = pd.DataFrame(df["awayteam"])
df_away.rename(columns = {"awayteam":'team'}, inplace = True)
df_home = pd.DataFrame(df["hometeam"])
df_home.rename(columns = {"hometeam":'team'}, inplace = True)
df_teams = df_away.append(df_home)

#Sort values
df_teams.sort_values(by="team", inplace = True)

#Drop duplicates
df_teams.drop_duplicates(subset ="team", keep = "first", inplace = True)

#Reset the index
df_teams = df_teams.reset_index(drop=True)

#Deletion of useless variables
del df_away
del df_home

#Clean file creation for analysis

os.makedirs('working_files', exist_ok=True)  
df_teams.to_csv('working_files/team_list_prep.csv') 

df_teams

C:\Users\itrem\AppData\Local\Temp\ipykernel_21060\2809839147.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_teams = df_away.append(df_home)


team
0        49ers
1        Bears
2      Bengals
3        Bills
4      Broncos
5       Browns
6   Buccaneers
7    Cardinals
8     Chargers
9       Chiefs
10       Colts
11     Cowboys
12    Dolphins
13      Eagles
14     Falcons
15      Giants
16     Jaguars
17        Jets
18       Lions
19     Packers
20    Panthers
21    Patriots
22     Raiders
23        Rams
24      Ravens
25      Saints
26    Seahawks
27    Steelers
28      Texans
29      Titans
30     Vikings
31  Washington